In [126]:
!pip install mne

import numpy as np
import scipy.io
import mne
import matplotlib.pyplot as plt
from google.colab import drive
from mne.preprocessing import ICA

In [127]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/BCI_Comp_III_Wads_2004.zip -d /content/bci2004

Archive:  /content/drive/MyDrive/BCI_Comp_III_Wads_2004.zip
replace /content/bci2004/BCI_Comp_III_Wads_2004/BCI_Comp_III_Wads_2004.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
file_path = '/content/bci2004/BCI_Comp_III_Wads_2004/Subject_A_Train.mat'
mat_data = scipy.io.loadmat(file_path)


In [ ]:
signal = mat_data['Signal']

epochs_num, samples_num, channels_num = signal.shape
signal = signal.reshape(epochs_num * samples_num, channels_num)
signal = (signal.T) * 1e-6

In [ ]:
mat_data.keys()

In [ ]:
flashing_marker = mat_data['Flashing'].flatten()
stimulus_marker = mat_data['StimulusCode'].flatten()
assert signal.shape[1] == flashing_marker.size == stimulus_marker.size

In [ ]:
events = []
for i in range(1, flashing_marker.size):
    if flashing_marker[i] == 1 and flashing_marker[i - 1] == 0:
        event_id = int(stimulus_marker[i])
        if event_id != 0:
            events.append([i, 0, event_id])

events = np.array(events, dtype=int)

In [ ]:
sfreq = 240
n_channels = signal.shape[0]
ch_names = [f'EEG{i+1}' for i in range(n_channels)]
ch_types = ['eeg'] * n_channels

info = mne.create_info(
    ch_names=ch_names,
    sfreq=sfreq,
    ch_types=ch_types
)

raw = mne.io.RawArray(signal, info)

In [ ]:
fig = raw.compute_psd().plot()

In [ ]:
raw_filtr = raw.copy().filter(l_freq=0.1, h_freq=20.0, fir_design='firwin')

In [ ]:
raw_filtr.resample(sfreq=120)

In [ ]:
fig = raw_filtr.compute_psd().plot()

In [ ]:
montage = mne.channels.make_standard_montage('standard_1020')
standard_names = montage.ch_names

n_rename = min(len(standard_names), len(raw_filtr.ch_names))

mapping = {}
for i in range(n_rename):
    mapping[raw_filtr.ch_names[i]] = standard_names[i]

raw_filtr.rename_channels(mapping)
raw_filtr.set_montage(montage, match_case=False)

In [ ]:
raw_ica = raw_filtr.copy().filter(l_freq=1.0, h_freq=None)
ica = ICA(n_components=15, max_iter="auto", random_state=97)
ica.fit(raw_ica, reject=dict(eeg=150e-6))
ica

In [ ]:
ica.plot_components()

In [ ]:
ica.exclude = [1, 3]

In [ ]:
raw_clean = raw_filtr.copy()
ica.apply(raw_clean)

In [ ]:
raw_clean.plot(n_channels=10, duration=5, title='Data AFTER removing Blinks')

In [ ]:
stimulus_type = mat_data['StimulusType'].flatten()
events = []
for i in range(1, flashing_marker.size):
    if flashing_marker[i] == 1 and flashing_marker[i - 1] == 0:
        if int(stimulus_type[i]) == 1:
            events.append([i, 0, 1])
        else:
            events.append([i, 0, 2])

events = np.array(events, dtype=int)

event_id = {
    'Target': 1,
    'Non-Target': 2
}

epochs = mne.Epochs(
    raw_filtr.copy().filter(l_freq=None, h_freq=15), events, event_id, tmin=-0.2, tmax=0.8, baseline=(-0.2, 0), preload=True, reject=None
)

evoked_target = epochs['Target'].average()
evoked_nontarget = epochs['Non-Target'].average()

mne.viz.plot_compare_evokeds(
    {'Target': evoked_target, 'Non-Target': evoked_nontarget},
    picks='Cz',
    combine=None,
    title='P300 at Cz'
)




In [ ]:
mne.viz.plot_compare_evokeds(
    {'Target': evoked_target, 'Non-Target': evoked_nontarget},
    picks='Pz',
    combine=None,
    title='P300 at Pz'
)